MODELO DE RECOMENDACION DE JUEGOS:


Se realizo dos modelos de recomendación, que generen una lista de 5 juegos ya sea ingresando el nombre del juego o el id de usuario.

En el primer caso, el modelo tiene una relación ítem-ítem, esto es porque se  toma un juego y en base a que tan similar es ese juego con el resto de los juegos se recomiendan similares. En el segundo caso, el modelo aplica un filtro usuario-juego, es decir, toma id de usuario, encuentra usuarios similares y se recomiendan ítemso juegos que a esos usuarios similares les gustaron.

In [22]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import pyarrow as pa
import os
import pyarrow.parquet as pq
import pandas as pd

Datos a utilizar: 

Se leen los datos preparados en el EDA y se convierten en dataframe para ser utilizados por el modelo.



In [3]:
csv_file_path = 'df_3.csv'
df = pd.read_csv(csv_file_path)
df

,user_id,item_name,rating
0,76561197970982479,Killing Floor,5
1,76561197970982479,Zeno Clash,3
2,76561197970982479,Metro 2033,5
3,js41637,Barbie™ Dreamhouse Party™,5
4,js41637,Euro Truck Simulator 2,5
...,...,...,...
42779,wayfeng,Counter-Strike: Global Offensive,3
42780,76561198251004808,Enclave,5
42781,72947282842,Counter-Strike: Global Offensive,3
42782,ApxLGhost,Counter-Strike: Global Offensive,5


PRIMERO : 

Creación de Matrices de Características: Se crea un dataframe que contiene los 'user_id' como indices, los juegos ('item_name') como columnas y como valores los ´rating' mediante pivot_table

In [4]:
piv = df.pivot_table(index=['user_id'], columns=['item_name'], values='rating')
piv

item_name,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,123 Slaughter Me Street,140,16 Bit Arena,200% Mixed Juice!,...,ibb & obb,inMomentum,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Beave-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zynxgameth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


SEGUNDO:

 Se normalizan los valores del df piv restando la media de las calificaciones de un usuario y luego dividir por la diferencia entre el valor máximo y mínimo de las calificaciones. Esto ajusta las calificaciones de un usuario de manera que estén centradas en cero y escaladas en función de su variabilidad. Los usuarios que solo dieron una calificación o han calificado todos los juegos de la misma manera serán eliminados durante el proceso de normalización ya que no aportan información útil para el modelo de recomendación 

In [5]:
# Normalización del dataframe 'piv'
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# Se borran las columnas que contienen solo cero o no tienen rating, se rellenan los vacios con 0 y se hace la transpuesta
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]
piv_norm

user_id,-2SV-vuLB-Kg,-GM-Dragon,-Mad-,-PRoSlayeR-,-SEVEN-,-_PussyDestroyer_-,0-3-0,00000000000000000001227,00True,01189958889189157253,...,zile98,zixwot,zombi_anon,zomgieee,zoozles,zrustz16,zuilde,zuzuga2003,zv_odd,zyr0n1c
item_name,,,,,,,,,,,,,,,,,,,,,
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
12 Labours of Hercules,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


TERCERO: 

A los datos de esta matriz normalizada se los convierte a un formato de matriz dispersa (sparse matrix) para reducir la memoria utilizada y mejorar la eficiencia en el manejo de grandes conjuntos de datos, esto porque la mayoría de los valores en la matriz son ceros. La matriz dispersa es un tipo de estructura de datos que almacena solo los valores distintos de cero junto con su ubicación en la matriz, en lugar de almacenar todos los valores de la matriz, incluso los ceros.

In [6]:
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

<2640x6255 sparse matrix of type '<class 'numpy.float64'>'
	with 23432 stored elements in Compressed Sparse Row format>

TERCERO: 

Se crean dos matrices de similitud utilizando la similitud del coseno(el cual es la similitud entre dos vectores en un espacio multidimensional. se utiliza para determinar cuán similares son dos conjuntos de datos o elementos, y se calcula utilizando el coseno del ángulo entre los vectores que representan esos datos o elementos.) para medir la similitud entre los juegos (item_similarity) y entre los usuarios (user_similarity).

In [7]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

CUARTO: 

Creacion de dataframes donde estan contenidos las matrices anteriores.

In [8]:
#item similarity df:
item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)

#user similarity df:
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

QUINTO : FUNCION DE RECOMENDACION SEGUN JUEGO: 

Se realiza la función que realice una recomendación de 5 juegos en función de un juego dado. Esta función toma un nombre de un juego como entrada, luego ordena la columna correspondiente a ese juego en la matriz de similitud entre elementos (item_sim_df) de manera descendente, de modo que los juegos más similares aparezcan en la parte superior. Posteriormente selecciona los 5 juegos más similares (excluyendo el propio juego que se pasó como entrada), itera a través de estos juegos similares y, finalmente, imprime una lista de juegos similares al juego especificado.


In [10]:
def top_game(game):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        None: Esta función imprime una lista de juegos 5 similares al dado.

    '''
    count = 1
    print('Similar games to {} include:\n'.format(game))
    for item in item_sim_df.sort_values(by = game, ascending = False).index[1:6]:
        print('No. {}: {}'.format(count, item))
        count +=1  

In [11]:
# ejemplo
top_game('theHunter')

Similar games to theHunter include:

No. 1: Hover : Revolt Of Gamers
No. 2: Zombie Exodus
No. 3: SNOW
No. 4: Grand Theft Auto III
No. 5: Half-Life 2: Lost Coast


SEXTO : FUNCION DE RECOMENDACION SEGUN USUARIO:  

Esta funcion  muestra una lista de usuarios más similares a un usuario dado, junto con sus valores de similitud. Primero, ordena la matriz de similitud entre usuarios (user_sim_df) en orden descendente según la similitud con el usuario dado ('user_id'), luego toma los 5 usuarios más similares (excluyendo el propio usuario) y almacena sus nombres y valores de similitud en las listas sim_users y sim_values. Finalmente, combina los nombres de usuario y los valores de similitud en una lista de tuplas utilizando la función zip y los imprime.


In [12]:
def top_users(user):
    '''
    Muestra una lista de los usuarios más similares a un usuario dado y sus valores de similitud.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean encontrar usuarios similares.

    Returns:
        None: Esta función imprime la lista de usuarios similares y sus valores de similitud en la consola.

    '''
    # Verificando si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users:\n')
    # Ordenando los usuarios por similitud descendente y toma los 5 usuarios más similares (excluyendo el propio 'user')
    sim_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:6]
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    # Combina los nombres de usuario y los valores de similitud en una lista de tuplas
    zipped = zip(sim_users, sim_values,)
    
    # Itera a través de las tuplas y muestra los usuarios similares y sus valores de similitud
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 

In [14]:
# ejemplo
top_users('zuilde')

Most Similar Users:

User #76561198097949523, Similarity value: 0.62
User #LoiroG, Similarity value: 0.41
User #killergamer1232, Similarity value: 0.40
User #imonyouutube, Similarity value: 0.40
User #KUYCAT, Similarity value: 0.39


In [13]:
#En esta función, se crea una lista de 5 juegos recomendados para un usuario en función de las calificaciones de usuarios similares.
# Los juegos que son más frecuentemente recomendados por usuarios similares se consideran como las principales recomendaciones para ese usuario.

def similar_user_recs(user):
    '''
    Genera una lista de los juegos más recomendados para un usuario, basándose en las calificaciones de usuarios similares.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean generar recomendaciones.

    Returns:
        list: Una lista de los juegos más recomendados para el usuario basado en la calificación de usuarios similares.

    '''
    # Verificando si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    # Obteniendo los usuarios más similares al usuario dado
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    
    best = []  # Lista para almacenar los juegos mejor calificados por usuarios similares
    most_common = {}  # Diccionario para contar cuántas veces se recomienda cada juego
    
    # Para cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista 'best'
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        best.append(piv_norm[piv_norm.loc[:, i]==max_score].index.tolist())
    
    # Cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    
    # Ordena los juegos por la frecuencia de recomendación en orden descendente
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    
    # Devuelve los 5 juegos más recomendados
    return sorted_list[:5]  

In [14]:
# ejemplo
similar_user_recs('00True')

[('DayZ', 10),
 ("Don't Starve", 1),
 ('Arma 3', 1),
 ('Squad', 1),
 ('Gotham City Impostors: Free To Play', 1)]

In [15]:
# ejemplo
similar_user_recs('76561198097949523')


[('Terraria', 6),
 ('Left 4 Dead 2', 4),
 ('Neverwinter', 1),
 ('Unturned', 1),
 ('Warframe', 1)]

SETIMO : Carga de las matrices para recomendar en la API

Para poder usar las funciones top_users y similar_user_recs es necesario consumir las matrices piv_norm y user_sim_df. Se guarda en formato parquet lo que permite una compresión y codificación eficiente.


In [20]:

# Creando directorio 'data' :
directorio_data = os.path.join(os.getcwd(), 'data')
if not os.path.exists(directorio_data):
    os.makedirs(directorio_data)

# Guardando el DataFrame piv_norm en Parquet
tabla_piv_norm = pa.Table.from_pandas(piv_norm)
pq.write_table(tabla_piv_norm, os.path.join(directorio_data, 'piv_norm.parquet'))

# Guardando el DataFrame item_sim_df en Parquet
tabla_item_sim_df = pa.Table.from_pandas(item_sim_df)
pq.write_table(tabla_item_sim_df, os.path.join(directorio_data, 'item_sim_df.parquet'))

# Guarda el DataFrame user_sim_df en Parquet
tabla_user_sim_df = pa.Table.from_pandas(user_sim_df)
pq.write_table(tabla_user_sim_df, os.path.join(directorio_data, 'user_sim_df.parquet'))

print("Los DataFrames se guardaron exitosamente en archivos Parquet en el directorio 'data'.")


Los DataFrames se guardaron exitosamente en archivos Parquet en el directorio 'data'.


In [21]:
pq.write_table(pa.Table.from_pandas(piv_norm), 'data/piv_norm.parquet')
pq.write_table(pa.Table.from_pandas(user_sim_df), 'data/user_sim_df.parquet')
pq.write_table(pa.Table.from_pandas(item_sim_df), 'data/item_sim_df.parquet')
print('Se guardaron correctamente')

Se guardaron correctamente


In [23]:
import pandas as pd
import pyarrow as pa

# Read the Parquet files into DataFrames
piv_norm_df = pd.read_parquet('data/piv_norm.parquet')
user_sim_df = pd.read_parquet('data/user_sim_df.parquet')
item_sim_df = pd.read_parquet('data/item_sim_df.parquet')

# Print the head of each DataFrame to confirm they were loaded correctly
print("DataFrame piv_norm:")
print(piv_norm_df.head())

print("\nDataFrame user_sim_df:")
print(user_sim_df.head())

print("\nDataFrame item_sim_df:")
print(item_sim_df.head())

# Now you have your DataFrames loaded into memory and can perform operations on them
# ...


DataFrame piv_norm:
user_id                 -2SV-vuLB-Kg  -GM-Dragon  -Mad-  -PRoSlayeR-  -SEVEN-  \
item_name                                                                       
0RBITALIS                        0.0         0.0    0.0          0.0      0.0   
10,000,000                       0.0         0.0    0.0          0.0      0.0   
100% Orange Juice                0.0         0.0    0.0          0.0      0.0   
1001 Spikes                      0.0         0.0    0.0          0.0      0.0   
12 Labours of Hercules           0.0         0.0    0.0          0.0      0.0   

user_id                 -_PussyDestroyer_-  0-3-0  00000000000000000001227  \
item_name                                                                    
0RBITALIS                              0.0    0.0                      0.0   
10,000,000                             0.0    0.0                      0.0   
100% Orange Juice                      0.0    0.0                      0.0   
1001 Spikes           